## 1. 远程仓库

到目前为止，我们已经掌握了如何在Git仓库里对一个文件进行时光穿梭，你再也不用担心文件备份或者丢失的问题了。

可是有用过集中式版本控制系统SVN的童鞋会站出来说，这些功能在SVN里早就有了，没看出Git有什么特别的地方。

没错，如果只是在一个仓库里管理文件历史，Git和SVN真没啥区别。为了保证你现在所学的Git物超所值，将来绝对不会后悔，同时为了打击已经不幸学了SVN的童鞋，本章开始介绍Git的杀手级功能之一（注意是之一，也就是后面还有之二，之三……）：远程仓库。

Git是分布式版本控制系统，同一个Git仓库，可以分布到不同的机器上。怎么分布呢？最早，肯定只有一台机器有一个原始版本库，此后，别的机器可以“克隆”这个原始版本库，而且每台机器的版本库其实都是一样的，并没有主次之分。

你肯定会想，至少需要两台机器才能玩远程库不是？但是我只有一台电脑，怎么玩？

其实一台电脑上也是可以克隆多个版本库的，只要不在同一个目录下。不过，现实生活中是不会有人这么傻的在一台电脑上搞几个远程库玩，因为一台电脑上搞几个远程库完全没有意义，而且硬盘挂了会导致所有库都挂掉，所以我也不告诉你在一台电脑上怎么克隆多个仓库。

实际情况往往是这样，找一台电脑充当服务器的角色，每天24小时开机，其他每个人都从这个“服务器”仓库克隆一份到自己的电脑上，并且各自把各自的提交推送到服务器仓库里，也从服务器仓库中拉取别人的提交。

完全可以自己搭建一台运行Git的服务器，不过现阶段，为了学Git先搭个服务器绝对是小题大作。好在这个世界上有个叫GitHub的神奇的网站，从名字就可以看出，这个网站就是提供Git仓库托管服务的，所以，只要注册一个GitHub账号，就可以免费获得Git远程仓库。

为什么GitHub需要SSH Key呢？因为GitHub需要识别出你推送的提交确实是你推送的，而不是别人冒充的，而Git支持SSH协议，所以，GitHub只要知道了你的公钥，就可以确认只有你自己才能推送。(这就涉及到了Digital Signature了, 既然是加密，那肯定是不希望别人知道我的消息，所以只有我才能解密，所以可得出公钥负责加密，私钥负责解密；同理，既然是签名，那肯定是不希望有人冒充我发消息，只有我才能发布这个签名，所以可得出私钥负责签名，公钥负责验证。Gihub通过我们的public key来验证我们是我们, 我们只需要用私钥签名)

当然，GitHub允许你添加多个Key。假定你有若干电脑，你一会儿在公司提交，一会儿在家里提交，只要把每台电脑的Key都添加到GitHub，就可以在每台电脑上往GitHub推送了。

最后友情提示，在GitHub上免费托管的Git仓库，任何人都可以看到喔（但只有你自己才能改）。所以，不要把敏感信息放进去。

如果你不想让别人看到Git库，有两个办法:

- 一个是交点保护费，让GitHub把公开的仓库变成私有的，这样别人就看不见了（不可读更不可写）。

- 另一个办法是自己动手，搭一个Git服务器，因为是你自己的Git服务器，所以别人也是看不见的。这个方法我们后面会讲到的，相当简单，公司内部开发必备。

参考:

- https://www.liaoxuefeng.com/wiki/896043488029600/896954117292416

## 2. Linux Commands
-----

- `touch` command is used to create a file without any content.

在Linux上创建一个新的用户`git`之后, 使用`su git`切换, 然后该user`git`并不能执行sudo权限, 如果你使用`sudo mkdir /srv/resp`类似这种需要最高权限的命令, 系统就会提示你:`git is not in the sudoers file.` 意思是说所以我们要查一下怎么把我们的user`git`添加到sudoers file. 

```
su root 
sudo vi /etc/sudoers
```

然后把`user_name ALL=(ALL)  ALL`加进上面的这个文件, 注意`user_name`要换成你的用户名, 这里就是`git`, 然后用户`git`就有sudo权限了.

```
-rw------- =  600
-rw-rw-rw- =  666
-rwxrwxrwx =  777
```

了解更多: https://www.linuxtopia.org/online_books/introduction_to_linux/linux_The_chmod_command.html

## 3. `/srv`
----

`/srv` contains site-specific data which is served by this system.

This main purpose of specifying this is so that users may find the location of the data files for particular service, and so that services which require a single tree for readonly data, writable data and scripts (such as cgi scripts) can be reasonably placed. **Data that is only of interest to a specific user should go in that users' home(`~/`) directory**. 

The methodology used to name subdirectories of `/srv` is unspecified as there is currently no consensus on how this should be done. One method for structuring data under `/srv` is by protocol, eg. `ftp`, `rsync`, `www`, and `cvs`. On large systems it can be useful to structure `/srv` by administrative context, such as `/srv/physics/www`, `/srv/compsci/cvs`, etc. This setup will differ from host to host. Therefore, no program should rely on a specific subdirectory structure of `/srv` existing or data necessarily being stored in `/srv`. However `/srv` should always exist on FHS compliant systems and should be used as the default location for such data. 

了解更多: https://tldp.org/LDP/Linux-Filesystem-Hierarchy/html/srv.html

## 3. `git init` & `git init --bare`
----

`git init --bare demo2`, 创建的demo2目录下的内容就和`git init demo1`中`demo1/.git`目录下的内容相同, 所以就相当于, demo2就是个`.git`文件, 而`.git`就是用来保存历史版本信息的, 所以demo2就没有工作目录了, 啥是工作目录?

比如你在你的项目`GreenhouseMonitor`里想版本控制, 你会输入`git init`, 这时候`GreenhouseMonitor`目录下有什么`src`, `webapp`等各种源代码, 然后还有个`.git`文件夹, 这个文件夹用来保存你git相关的配置信息和历史版本信息. 那些`src`, `webapp`就是工作目录. 

```shell
# git init demo1
# cd demo1
# ls demo1/.git 
HEAD        config      description hooks       info        objects     refs

# git init --bare demo2
# ls deme2 
HEAD        config      description hooks       info        objects     refs
```


## 4. Git服务器
-----

### 4.1. 添加Public Key

在远程仓库一节中，我们讲了远程仓库实际上和本地仓库没啥不同，纯粹为了7x24小时开机并交换大家的修改。

GitHub就是一个免费托管开源代码的远程仓库。但是对于某些视源代码如生命的商业公司来说，既不想公开源代码，又舍不得给GitHub交保护费，那就只能自己搭建一台Git服务器作为私有仓库使用。搭建Git服务器需要准备一台运行Linux的机器. 

在root用户下执行以下操作, 

```
cd

mkdir .ssh && chmod 700 .ssh

touch .ssh/authorized_keys && chmod 600 .ssh/authorized_keys
```

Next, you need to add some developer SSH public keys to the `authorized_keys` file for the git user. Let’s assume you have some trusted public keys and have saved them to temporary files. 就是把你本地电脑的ssh公钥放到这, 这就是让服务器知道你是你, Digital Signature机制, 下面指令是追加文字的:

```
cat a.txt >> ~/.ssh/authorized_keys
```

### 4.2. 在服务器创建 Bare Git Repository

Now, you can set up an empty repository for them by running `git init --bare`, which initializes the repository without a working directory:

```
cd /srv/git
sudo mkdir blogs.git
cd blogs.git/
sudo git init --bare
Initialized empty Git repository in /srv/git/blogs.git/
```

OK, 你的远程仓库设置好了, 注意一般我们创建bare repository时, 习惯命名为`xxx.git`, 然后在本地创建真正工作的项目, 名字要和`xxx.git`中的前缀一样, 就是`xxx`就行了, 别忘了远程仓库其实github,

### 4.3. 在本地创建本地仓库

```
# 在本地创建的项目名字是blogs而不是blogs.git或其他名字
git init blogs

vi a.txt

git add .

git commit -m "add a.txt"

# 注意看这个命令的参数 root是linux的用户 /srv/git/blogs.git是路径
git remote add origin root@shaowenzhu.top:/srv/git/blogs.git

git push origin main      
```

成功, 想一下你现在可以直接提交成功就是因为上一步你在`~/.ssh/authorized_keys`添加了你本地电脑的publi key, 和github添加public key一个道理, 现在你已经把修改提交到远程服务器了, 这样你就可以在其他电脑, 或者在本地电脑的另一个文件夹克隆这个仓库了, 下面看看怎么克隆, 克隆之后得到的是什么. 

### 4.4. 远程克隆仓库

```
git clone root@shaowenzhu.top:/srv/git/blogs.git

# 看, 克隆后的仓库名字为blogs, 并不是blogs.git
cd blogs

# 文件夹里有了刚我们提交的a.txt文件
ls -a
.     ..    .git  a.txt

# 看看都是啥
git show
commit d3d81ab86d71a38405d8ff54fdc2fa612856122b (HEAD -> main, origin/main)
Author: Shaowen Zhu <shaowen@macbook-11.home>
Date:   Fri Apr 14 09:02:41 2023 -0300

    add a.txt file

diff --git a/a.txt b/a.txt
new file mode 100644
index 0000000..a042389
--- /dev/null
+++ b/a.txt
@@ -0,0 +1 @@
+hello world!
```

参考: https://git-scm.com/book/en/v2/Git-on-the-Server-Setting-Up-the-Server